## Task 1

In [68]:
import pandas_gbq
import os
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import storage
from google.cloud.exceptions import NotFound
import numpy as np
import sqlite3
import zipfile
import csv


In [69]:

# Paths
file_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded"

# Read CSVs in chunks and upload to BigQuery
credentials_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project-bt-b14310631abc.json"
credentials = service_account.Credentials.from_service_account_file(credentials_path)
gbq_proj_id = "wedge-project-bt"
dataset_id = "wedge-data"



# # Read CSVs in chunks and upload to BigQuery
# credentials_path = r"C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\umt-msba-gg-key.json"
# credentials = service_account.Credentials.from_service_account_file(credentials_path)
# gbq_proj_id = "umt-msba"
# dataset_id = "wedge_transactions"




In [ ]:
# Define the directory where your zip files are located
zip_dir = "C:\\Users\\britt\\OneDrive - The University of Montana\\Applied Data Analytics\\Wedge Project\\WedgeZipOfZips"

# Define the directory where you want to save the unzipped files
extract_to_dir = "C:\\Users\\britt\\OneDrive - The University of Montana\\Applied Data Analytics\\Wedge Project\\wedge-project\\Uploaded"

# Walk through the directory
for root, dirs, files in os.walk(zip_dir):
    for file in files:
        if file.endswith('.zip'):
            # Construct the file path
            file_path = os.path.join(root, file)
            # Open the zip file
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                # Extract all the contents into the directory
                zip_ref.extractall(extract_to_dir)
                print(f"Extracted {file} to {extract_to_dir}")

print("All files have been extracted.")


In [ ]:
# Headers for the files
headers = [
    'datetime', 'register_no', 'emp_no', 'trans_no', 'upc', 'description', 'trans_type', 'trans_subtype', 'trans_status',
    'department', 'quantity', 'Scale', 'cost', 'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp',
    'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 'percentDiscount', 'ItemQtty', 'volDiscType',
    'volume', 'VolSpecial', 'mixMatch', 'matched', 'memType', 'staff', 'numflag', 'itemstatus', 'tenderstatus', 'charflag',
    'varflag', 'batchHeaderID', 'local', 'organic', 'display', 'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id'
]

# Loop through all files in the directory
for root, dirs, files in os.walk(file_path):
    for file in files: 
        full_path = os.path.join(root, file) 
        if file.endswith('.csv'): 
            with open(full_path, 'r') as f: 
                first_line = f.readline().strip()

            # Check if the file likely has headers based on the first line
            if first_line.startswith('"datetime"') or first_line.startswith('datetime'):
                print(f"File {file} seems to already have headers. Skipping...")
                continue

            # If not, then prepend headers to the file
            print(f"Adding headers to {file}")
            with open(full_path, 'r') as f:
                content = f.read()
            with open(full_path, 'w') as f:
                f.write(','.join(headers) + '\n' + content)


In [73]:
chunk_size = 50000  # Define the chunk size for reading the CSV



# Define the function to drop a table if it exists
def drop_table_if_exists(dataset_id, table_name, credentials, gbq_proj_id):
    client = bigquery.Client(credentials=credentials, project=gbq_proj_id)
    table_id = f"{gbq_proj_id}.{dataset_id}.{table_name}"
    try:
        client.delete_table(table_id)  # API request
        print(f"Deleted table '{table_id}'")
    except NotFound:
        print(f"Table '{table_id}' not found, skipping deletion.")

# Define the detect_delimiter function
def detect_delimiter(filename):
    with open(filename, 'r') as file:
        first_line = file.readline()
        return ";" if ";" in first_line else ","

# Define the clean_dataframe function
def clean_dataframe(df):
    # Type Conversion lists
    float_columns = [
        'register_no', 'emp_no', 'trans_no', 'department', 'quantity', 'Scale', 'cost', 
        'unitPrice', 'total', 'regPrice', 'altPrice', 'tax', 'taxexempt', 'foodstamp', 
        'wicable', 'discount', 'memDiscount', 'discountable', 'discounttype', 'voided', 
        'percentDiscount', 'ItemQtty', 'volDiscType', 'volume', 'VolSpecial', 'mixMatch', 
        'matched', 'numflag', 'itemstatus', 'tenderstatus', 'varflag', 'local', 'organic', 
        'receipt', 'card_no', 'store', 'branch', 'match_id', 'trans_id'
    ]
    boolean_columns = ['memType', 'staff', 'batchHeaderID', 'display']
    string_columns = ['upc', 'description', 'trans_type', 'trans_subtype', 'trans_status', 'charflag']

    # Convert string columns first
    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)
    # Trim spaces from string columns and replace double quotes
    df[col] = df[col].str.replace('"', '', regex=False)

    # Clean and convert float columns
    for col in float_columns:
        if col in df.columns:
            # Replace non-numeric characters with NaN and convert to float
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Handle missing values in float columns, for example, by filling with 0
    df[float_columns] = df[float_columns].fillna(0)

    # Convert boolean columns
    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].astype(bool)

   # Convert datetime
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')


    # Replace specific strings with an empty string
    replace_strings = ["\\N", "\\\\N", "nan", "NULL"]
    df.replace(replace_strings, "", inplace=True)

    # Trim spaces from string columns and replace double quotes
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
            df[col] = df[col].str.replace('\\\"', '', regex=False)

    # Replace empty strings with None
    df = df.applymap(lambda x: None if x == '' else x)


    return df
    

print(f"Contents of {file_path}:")
print(os.listdir(file_path))


# Loop through all files in the directory
for root, dirs, files in os.walk(file_path):
    for file in files:
        full_path = os.path.join(root, file)

        if file.endswith('.csv'):
            print(f"Found CSV file: {file}")

            delimiter = detect_delimiter(full_path)
            print(f"Detected delimiter: {delimiter}")

            # Reading CSV file with correct handling of quoted fields
            chunk_iter = pd.read_csv(full_path, delimiter=delimiter, quotechar='"', chunksize=chunk_size, dtype=str, low_memory=False)

            table_name = file.replace('.csv', '')

            # Drop the table if it exists
            drop_table_if_exists(dataset_id, table_name, credentials, gbq_proj_id)

            print(f"Reading CSV file in chunks: {file}...")
            for idx, chunk_df in enumerate(chunk_iter):
                # Clean the dataframe
                chunk_df = clean_dataframe(chunk_df)

                # Modify the field names to comply with BigQuery rules
                chunk_df.columns = [col.lower().replace(';', '') for col in chunk_df.columns]

                print(f"Uploading chunk {idx + 1} to {table_name}...")
                if idx == 0:
                    # For the first chunk, create the table with the defined schema
                    pandas_gbq.to_gbq(chunk_df, f"{dataset_id}.{table_name}", project_id=gbq_proj_id, if_exists='replace', credentials=credentials, table_schema=schema)
                else:
                    # For subsequent chunks, append to the table
                    pandas_gbq.to_gbq(chunk_df, f"{dataset_id}.{table_name}", project_id=gbq_proj_id, if_exists='append', credentials=credentials)
                del chunk_df  # Clear the chunk from memory


Contents of C:\Users\britt\OneDrive - The University of Montana\Applied Data Analytics\Wedge Project\wedge-project\Uploaded:
['transArchive_201110_201112.csv', 'transArchive_201201_201203.csv', 'transArchive_201201_201203_inactive.csv', 'transArchive_201204_201206.csv', 'transArchive_201204_201206_inactive.csv', 'transArchive_201207_201209.csv', 'transArchive_201207_201209_inactive.csv', 'transArchive_201210_201212.csv', 'transArchive_201210_201212_inactive.csv', 'transArchive_201301_201303.csv', 'transArchive_201301_201303_inactive.csv', 'transArchive_201304_201306.csv', 'transArchive_201304_201306_inactive.csv', 'transArchive_201307_201309.csv', 'transArchive_201307_201309_inactive.csv', 'transArchive_201310_201312.csv', 'transArchive_201310_201312_inactive.csv', 'transArchive_201401_201403.csv', 'transArchive_201401_201403_inactive.csv', 'transArchive_201404_201406.csv', 'transArchive_201404_201406_inactive.csv']
Found CSV file: transArchive_201110_201112.csv
Detected delimiter: ,
D

C:\Users\britt\AppData\Local\Temp\ipykernel_22832\1431577111.py:71: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if x == '' else x)


Uploading chunk 1 to transArchive_201110_201112...


100%|██████████| 1/1 [00:00<?, ?it/s]
C:\Users\britt\AppData\Local\Temp\ipykernel_22832\1431577111.py:71: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if x == '' else x)


Uploading chunk 2 to transArchive_201110_201112...


...

100%|██████████| 1/1 [00:00<00:00, 994.38it/s]


Found CSV file: transArchive_201404_201406_inactive.csv
Detected delimiter: ;
Deleted table 'wedge-project-bt.wedge_data.transArchive_201404_201406_inactive'
Reading CSV file in chunks: transArchive_201404_201406_inactive.csv...


C:\Users\britt\AppData\Local\Temp\ipykernel_22832\1431577111.py:71: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: None if x == '' else x)


Uploading chunk 1 to transArchive_201404_201406_inactive...


100%|██████████| 1/1 [00:00<?, ?it/s]
